### Searching for activity ID

In [19]:
# Change this to be your API key.
MY_API_KEY="7SAZBY9HVTMZBTGQECC9J7SB0TAY"

url = "https://beta3.api.climatiq.io/search"

# ask user for query
query= input("Enter query: ")

query_params = {
    # Free text query can be writen as the "query" parameter
    "query": query,
    # You can also filter on region, year, source and more
    "region": "US"
}

# You must always specify your AUTH token in the "Authorization" header like this.
authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

# This performs the request and returns the result as JSON
response = requests.get(url, params=query_params, headers=authorization_headers).json()

# And here you can do whatever you want with the results
total = response['total_results']
print("There are", total, "results")

# save results as a list
results = response['results']

# run through and list all the activity ids from results
for index in range(len(results)):
    print(results[index]['activity_id'])

Enter query: grid mix
There are 13 results
electricity-energy_source_grid_mix_residual_mix_market_based
electricity-energy_source_grid_mix
electricity-energy_source_grid_mix
electricity-energy_source_grid_mix
electricity-energy_source_grid_mix
electricity-energy_source_grid_mix
electricity-energy_source_grid_mix
electricity-energy_source_grid_mix_location_based
electricity-energy_source_grid_mix
electricity-energy_source_grid_mix
electricity-energy_source_grid_mix
electricity-energy_source_grid_mix_losses
electricity-energy_source_grid_mix_non_baseload


### Estimating Electricity

In [20]:
import json
import requests

# your API key
MY_API_KEY="7SAZBY9HVTMZBTGQECC9J7SB0TAY"

url = "https://beta3.api.climatiq.io/estimate"

# The activity ID for the emission factor
activity_id = "electricity-energy_source_grid_mix"

# We have many regions with the same activity id, representing the power grid in different countries.
region = "US"
year = "2022"

# Different emission factors have different requirement as to what units they support estimates for.
energy_input = int(input("Enter the amount of hours KW that was used this month: "))  # user input from front-end
parameters = {
    "energy": energy_input,
    "energy_unit": "kWh"
    }

json_body = {
    "emission_factor": {
        "activity_id": activity_id,
        "region": region,
    },
    # Specify how much energy we're estimating for
    "parameters": parameters
}

# You must always specify your AUTH token in the "Authorization" header like this.
authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

# This line will send the request and retrieve the body as JSON.
response = requests.post(url, json=json_body, headers=authorization_headers).json()

# You can now do anything you want with the response
electric_co2 = response['co2e']
print(f'{electric_co2:.2f} kg')
electric_base = 351.87

Enter the amount of hours KW that was used this month: 943
351.87 kg


### Estimating Gas

In [21]:
import json
import requests

# your API key
MY_API_KEY="7SAZBY9HVTMZBTGQECC9J7SB0TAY"

url = "https://beta3.api.climatiq.io/estimate"

# The activity ID for the emission factor - determined by user
choice = int(input("Enter '1' for passenger vehicle. '2' for pickup\SUV, or '3' for motorcycle:  "))
if choice == 1:
    activity_id = "passenger_vehicle-vehicle_type_automobile-fuel_source_na-distance_na-engine_size_na"
    region = "Global"
    year = "2017"
elif choice == 2:
    activity_id = "passenger_vehicle-vehicle_type_pickup_trucks_vans_suvs-fuel_source_na-engine_size_na-vehicle_age_na-vehicle_weight_na"
    region = "US"
    year = "2022"
elif choice == 3:
    activity_id = "passenger_vehicle-vehicle_type_motorcycle-fuel_source_na-engine_size_na-vehicle_age_na-vehicle_weight_na"
    region = "US"
    year = "2021"
else:
    print("Incorrect selection")

# Different emission factors have different requirement as to what units they support estimates for.
if choice != 2:
    distance_input = float(input("Enter the amount of miles driven this month: "))  # user input from front-end
    distance_unit = "mi"
    if choice == 1:
        distance_input = distance_input * 1.609  # convert to KM
        distance_unit = "km"
    parameters = {
        "distance": distance_input,
        "distance_unit": distance_unit
        }

    json_body = {
        "emission_factor": {
            "activity_id": activity_id,
            "region": region,
        },
        # Specify how much energy we're estimating for
        "parameters": parameters
    }
elif choice == 2:
    money_input = float(input("Enter the amount of money (in USD) spent on gas this month: "))
    parameters = {
        "money": money_input,
        "money_unit": "usd"
        }

    json_body = {
        "emission_factor": {
            "activity_id": activity_id,
            "region": region,
        },
        # Specify how much energy we're estimating for
        "parameters": parameters
    }
    
# You must always specify your AUTH token in the "Authorization" header like this.
authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

# This line will send the request and retrieve the body as JSON.
response = requests.post(url, json=json_body, headers=authorization_headers).json()

# Print monthly Co2 emmissions
gas_co2 = response["co2e"]
print(f'{gas_co2:.2f} kg')

# Compare with average
gas_base = 396.29   # kg per month - determined from 2009 EPA average annual emmissions
if gas_co2 > gas_base:
    print("You are over the average.")
elif gas_co2 < gas_base:
    print("You are below the average.")
else:
    print("You are at the average.")


Enter '1' for passenger vehicle. '2' for pickup\SUV, or '3' for motorcycle:  2
Enter the amount of money (in USD) spent on gas this month: 1000
327.00 kg
You are below the average.
